In [4]:
import pandas as pd
import numpy as np
import math

## Bridges 
Only the columns needed for the data in the model were used. 
To clean the duplicate bridges, a filter has been used. 
The bridges on the N1 and N2 roads were included.
The bridges were given a certain label according to the length of the bridge. 
When the bridges had a distance smaller to the distance of the previous bridge then their own length, the bridges were dropped. To exlcude very small bridges. 
The bridges were given a model type 'bridge' to identify the bridges. 

In [5]:
# Read the bridges file 
bridges = pd.read_excel('../data/BMMS_overview.xlsx')

In [33]:
# Keep only useful columns
keep_columns = ['road', 'lat', 'lon', 'length', 'condition', 'name', 'LRPName', 'chainage', 'km']
bridges = bridges[keep_columns]

# Filters out all variations of (R), (Right), (  R  ), etc.
patternDel = "\(\s*[a-zA-Z]*(R|r)[a-zA-Z]*\s*\)"
filter = bridges['name'].str.contains(patternDel)
bridges = bridges[~filter]

In [34]:
# Include bridges on N1 and N2
bridges = bridges[(bridges['road'].str.contains(pat = ('N1'), regex = False)) | (bridges['road'].str.contains(pat = ('N2'), regex = False))]

In [35]:
# Define bridge length bins and labels
length_bins = [0, 10, 50, 200, math.inf]
length_labels = ['S', 'M', 'L', 'XL']

# Categorize bridges per length class
bridges["length_class"] = pd.cut(bridges["length"], bins=length_bins, include_lowest=False, right=False, labels=length_labels)

# Calculate distance between bridges
bridges1 = bridges.copy()
bridges1["dist"] = bridges1["km"].diff()

# Drop all bridges that are closer to the previous bridge then their own length
bridges = bridges.loc[bridges1["dist"]*1000 >= bridges["length"]]

In [36]:
# Give bridges a model type
bridges['model_type'] = 'bridge'

In [37]:
bridges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1718 entries, 2 to 19507
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   road          1718 non-null   object  
 1   lat           1718 non-null   float64 
 2   lon           1718 non-null   float64 
 3   length        1718 non-null   float64 
 4   condition     1718 non-null   object  
 5   name          1718 non-null   object  
 6   LRPName       1718 non-null   object  
 7   chainage      1718 non-null   float64 
 8   km            1718 non-null   float64 
 9   length_class  1718 non-null   category
 10  model_type    1718 non-null   object  
dtypes: category(1), float64(5), object(5)
memory usage: 149.5+ KB


## Roads
Only the usefull columns for the model were included. The length of the roads have been calculated in meters. Because the bridges are also measured in meters. The roads were given a model type 'link' to identify the roads.
The N1 and N2 roads have been included as well as their side roads longer than 25 km. 
The first and last roads/links of the N1 and N2 were given the model type 'sourcesink' which indicated the begin and end of the road network. 
The model type 'intersection' is given to the intersection of the sideroads to the 'big' N1 and N2 roads. 

In [38]:
# Read the roads file
road = pd.read_csv('../data/_roads3.csv')

# Rename column 'lrp'
road = road.rename(columns = {'lrp':'LRPName'})

# Include usefull columns
road = road[["road", "chainage", "LRPName", "lat", "lon", "name"]]

# Calculate length of the roads in meters, drop the last (dummy) one
length_road = (road['chainage'] - road['chainage'].shift(-1)).abs()
road['length'] = (length_road * 1000)
road.drop(road.tail(1).index, inplace=True)
road.reset_index()

# Give roads a model type
road['model_type'] = 'link'

In [39]:
# Include roads on N1 and N2
road = road[(road['road'].str.contains(pat = ('N1'), regex = False)) | (road['road'].str.contains(pat = ('N2'), regex = False))]

# Identify roads smaller than 25 km and drop them
small_roads = road[(road['length'] < 25) & (road['road'] != 'N1') & (road['road'] != 'N2')].index
road = road.drop(small_roads)

In [40]:
# Creating list of sideroads and delete N1 and N2 
roads_list = road['road'].unique()
roads_list = np.delete(roads_list, 0)
roads_list = np.delete(roads_list, 17)
roads_list

array(['N101', 'N102', 'N103', 'N104', 'N105', 'N106', 'N107', 'N108',
       'N109', 'N110', 'N111', 'N112', 'N119', 'N120', 'N123', 'N128',
       'N129', 'N203', 'N204', 'N205', 'N206', 'N207', 'N208', 'N209',
       'N210'], dtype=object)

In [41]:
# The first road of the sideroad is the intersection of the sideroad with the N1 or N2 and are given model type 'intersection'
first_road = []
for first in roads_list:
    first_road.append(road[road['road'] == first].iloc[:1].index.values.tolist())    
for first1 in first_road:
    road.at[first1,'model_type'] = 'intersection'
    
# The last road of the sideroad is given the model type 'sourcesink' 
last_road = []
for last in roads_list:
    last_road.append(road[road['road'] == last].iloc[-1:].index.values.tolist())    
for last1 in last_road:
    road.at[last1,'model_type'] = 'sourcesink'

In [42]:
# Give N1 roads with LRPName LRPS or LRPE model type 'sourcesink'
road.loc[(road.road == 'N1') & (road.LRPName == 'LRPS'), ['model_type']] = 'sourcesink'
road.loc[(road.road == 'N1') & (road.LRPName == 'LRPE'), ['model_type']] = 'sourcesink'

# Give N2 roads with LRPName LRPS or LRPE model type 'intersection' or 'sourcesink'
road.loc[(road.road == 'N2') & (road.LRPName == 'LRPS'), ['model_type']] = 'intersection'
road.loc[(road.road == 'N2') & (road.LRPName == 'LRPE'), ['model_type']] = 'sourcesink'

# All intersections of the sideroads with N1 and N2 are given the model type 'intersection'
change_list = ['N109','N129','N123','N120','N102','N104','N207','N208','N206','N103','N210','N205','N107','N204','N103','N209', 'N205']
for i in change_list:
    road.loc[(road.road == i) & (road.LRPName == 'LRPE'), ['model_type']] = 'intersection'
    
# Give some roads manually a model type 'intersenction' or 'sourcesink'
road.loc[(road.road == 'N119') & (road.LRPName == 'LRP002a'), ['model_type']] = 'intersection'
road.loc[(road.road == 'N104') & (road.LRPName == 'LRPE'), ['model_type']] = 'sourcesink'
road.loc[(road.road == 'N107') & (road.LRPName == 'LRPS'), ['model_type']] = 'sourcesink'

## Mergen

The roads and bridges dataframes were merged using outer merging and sorted to get the rigth order.
The 'id' column has been added for the implementation to the PyCharm model.
The cells in the road column or the intersection with the N1 and N2 were given the corresponding 'N1' or 'N2'.
Duplicated intersection were given the same 'id' so the truckt know which road belongs to which intersection.
Than the dateframes are merged again and the file is saved. 

In [43]:
# Merge the roads and bridges dataframe
merge = pd.merge(road, bridges, how = 'outer', on = ["road", "LRPName", "length", "lat", "lon", "model_type", "chainage", "name"])

# Sort merge on 'lat' and 'lon'
merge = merge.sort_values(by = ['lat', 'lon'])
merge.reset_index()

,index,road,chainage,LRPName,lat,lon,name,length,model_type,condition,km,length_class
0,1338,N1,462.254,LRPE,20.862917,92.298083,"End of Road at Shapla Chattar ,Teknaf Meet wit...",462254.0,sourcesink,NaN,NaN,NaN
1,1337,N1,462.124,LRP467,20.862972,92.298083,Infor.missing,130.0,link,NaN,NaN,NaN
2,1336,N1,461.946,LRP466c,20.864667,92.298194,Bridge end,178.0,link,NaN,NaN,NaN
3,1335,N1,461.904,LRP466b,20.865028,92.298250,Bridge start,42.0,link,NaN,NaN,NaN
4,1334,N1,461.476,LRP466a,20.868860,92.298222,Box culvert,428.0,link,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5344,4424,N2,282.360,LRP283c,25.179803,92.033553,.,2.0,bridge,A,282.36,S
5345,4423,N2,282.270,LRP283b,25.180031,92.034177,.,3.5,bridge,A,282.27,S
5346,3132,N2,278.943,LRP280,25.180110,92.063778,Jaflong 8 km,887.0,link,NaN,NaN,NaN
5347,4422,N2,281.840,LRP282a,25.180537,92.037383,.,3.5,bridge,A,281.84,S


In [44]:
# Add 'id'column
merge.insert(0, 'id', range(0, len(merge)))
merge = merge[["id", "road", "name", "lat", "lon", "model_type", "chainage", "LRPName", "length", "condition"]]

In [45]:
# Merge on the rows where 'intersection' are the same
intersection = merge[merge['model_type'] == 'intersection']

In [46]:
# The cells in the road column of the intersections with N1 are called 'N1'
intersect_list = intersection[(intersection['name'].str.contains(pat = ('N1'), regex = False))].index
intersect_list
for y in intersect_list:
    intersection.at[y,'road'] = 'N1'

In [47]:
# The cells in the road column of the intersections with N1 are called 'N1'
intersect_listN2 = intersection[(intersection['name'].str.contains(pat = ('N2'), regex = False))].index
intersect_listN2
for y in intersect_listN2:
    intersection.at[y,'road'] = 'N2'

In [48]:
# Duplicate same intersection points to get same 'id'
intersection['road'][2176] = 'N1'
intersection['road'][1547] = 'N2'
intersection['road'][1559] = 'N1'
intersection['road'][1784] = 'N1'
intersection['road'][3418] = 'N207'
intersection['road'][1558] = 'N102'
intersection['road'][1548] = 'N102'
intersection['road'][3254] = 'N208'
intersection['road'][3631] = 'N208'
intersection['road'][3249] = 'N2'
intersection['road'][2187] = 'N1'
intersection['road'][1339] = 'N120'

<ipython-input-48-aafe8d6d3bbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection['road'][2176] = 'N1'
C:\Users\marlo\.anaconda\Python anaconda 3.8\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\marlo\.anaconda\Python anaconda 3.8\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [49]:
# Final merge on the same columns 
final_merged = pd.merge(merge, intersection, how = 'outer', on = ["id", "road", "LRPName", "length", "lat", "lon", "model_type", "chainage", "name", "condition"])
final_merged = final_merged.sort_values(by = ['lat', 'lon'])
final_merged.reset_index()
final_merged = final_merged[["id", "road", "name", "lat", "lon", "model_type", "chainage", "LRPName", "length", "condition"]]

In [50]:
final_merged.to_csv("../data/A3_data_clean.csv", index = False)